# Train Diffusion Model
### Imports

In [1]:
#!pip install librosa
#Set Dir 
import sys, os
sys.path.append(os.path.abspath('..'))

# Torch
import torch
from torch import nn, Tensor
from torch.utils.data import DataLoader
import torch.optim as optim
# Utils
import numpy as np
from numpy import ndarray
import logging

# Base Scripts
from Libraries.U_Net import *
from Libraries.Diffusion import *
from Libraries.Utils import *
from MainScripts.Conf import *

### Config
General

In [ ]:
remote_kernel: bool = True

logging_level: int = logging.INFO
model_name: str = "diffusion_v1"
model_path: str = path_to_remote_path(f"{MODEL_PATH}/{model_name}.pth", remote_kernel)
checkpoint_freq: int = 5 #0 for no checkpoint saving
training_data_name: str = "training_2000_1_1"

device = "cuda" if torch.cuda.is_available() else "cpu"

batch_size: int = 8
epochs: int = 100
diffusion_timesteps: int = 500

logging.basicConfig(level=logging_level, format='%(asctime)s - %(levelname)s - %(message)s')
logger: logging.Logger = logging.getLogger(__name__)

U-Net

In [3]:
learning_rate: float = 1e-4
n_starting_filters: int = 24
n_blocks: int = 2 #Each samples down by factor of 2
n_groups: int = 8 #For group norm
time_embed_dim: int = 64

### Data Loading

In [4]:
file: ndarray = load_training_data(path_to_remote_path(f"{DATA_PATH}/{training_data_name}.npy", remote_kernel))[:200, ...]

2025-03-03 13:07:32,038 - LIGHT_DEBUG - Ndarray loaded from Data/training_2000_1_1.npy of shape: (2000, 1024, 672)


In [5]:
data_loader = create_dataloader(Audio_Data(file), batch_size)
logger.info(f"Data loaded with shape: {file.shape}")

2025-03-03 13:07:32,364 - INFO - Data loaded with shape: (200, 1024, 672)


### Model Creation
U-Net

In [6]:
u_net = U_NET(in_channels=1, device=device, input_shape=[0, 0, file.shape[-2], file.shape[-1]], n_res_layers=n_blocks, n_starting_filters=n_starting_filters, n_groups=n_groups, time_emb_dim=time_embed_dim).to(device)
u_net = nn.DataParallel(u_net)
if os.path.exists(model_path):
    u_net.load_state_dict(torch.load(model_path, weights_only=False, map_location=device))
    logger.info(f"Model {model_name} loaded with {count_parameters(u_net)} Parameters")
else: 
    logger.info(f"Model {model_name} created with {count_parameters(u_net)} Parameters")

optimizer = optim.AdamW(u_net.parameters(), lr=learning_rate, weight_decay=0.001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

u_net = u_net.to(device)

#summary(u_net, [(1, 1024, 672), (1, 1, 1)])

2025-03-03 13:07:37,018 - INFO - Model diffusion_v1 created with 452065 Parameters


Diffusion

In [10]:
diffusion = Diffusion(u_net=u_net, u_net_optimizer=optimizer,diffusion_timesteps=diffusion_timesteps)
noise_schedule = cosine_noise(T=diffusion_timesteps).to(device)
#diffusion.visualize_diffusion_steps(x=torch.Tensor(file[:1]), noise_schedule=noise_schedule, device=device, n_spectograms=5)

### Train

In [ ]:
diffusion.train(data_loader, device, epochs=epochs, loss_function=U_Net_loss, noise_schedule=noise_schedule, checkpoint_freq=checkpoint_freq, model_path=model_path)

2025-03-03 13:08:52,000 - LIGHT_DEBUG - Batch 25/25


2025-03-03 13:08:52,699 - INFO - Epoch 01: Avg. Loss: 1.36904e-01 Remaining Time: 00h 47min 27s


2025-03-03 13:09:15,000 - LIGHT_DEBUG - Batch 25/25


2025-03-03 13:09:15,568 - INFO - Epoch 02: Avg. Loss: 1.27605e-01 Remaining Time: 00h 42min 09s


2025-03-03 13:09:38,000 - LIGHT_DEBUG - Batch 25/25


...

2025-03-03 13:27:42,000 - LIGHT_DEBUG - Batch 25/25


2025-03-03 13:27:42,522 - INFO - Epoch 50: Avg. Loss: 8.99571e-02 Remaining Time: 00h 19min 18s
2025-03-03 13:27:42,532 - LIGHT_DEBUG - Checkpoint saved model to Models/diffusion_v1.pth


2025-03-03 13:28:05,000 - LIGHT_DEBUG - Batch 25/25


2025-03-03 13:28:05,631 - INFO - Epoch 51: Avg. Loss: 9.46466e-02 Remaining Time: 00h 18min 55s


2025-03-03 13:28:16,000 - LIGHT_DEBUG - Batch 12/25